In [16]:
# Standard library imports
import json
import sys
from pathlib import Path
from enum import Enum
from typing import List

# Third party imports
import instructor
from instructor.function_calls import openai_schema
from openai import OpenAI

# Add parent directory to Python path to import models
sys.path.append(str(Path().absolute().parent))

# Local application imports
from models.input_models import GuardRail
from models.output_models import get_classes_with_enum
from prompts.format_structured_reasoning_user_prompt import format_structured_reasoning_user_prompt

# Load Utils
from utils.build_mermaid_diagram import build_mermaid_diagram
from utils.make_gpt_pro_prompt import make_prompt_for_gpt_prompt

ImportError: cannot import name 'Enum' from 'typing' (/Users/Ford/anaconda3/envs/promo/lib/python3.12/typing.py)

In [6]:
client = instructor.from_openai(OpenAI())

with open("../prompts/in_depth_thinking_system_prompt.md", "r") as f:
    IN_DEPTH_THINKING_SYSTEM_PROMPT = f.read()

### Show System Prompt (Optional)

In [5]:
# print(IN_DEPTH_THINKING_SYSTEM_PROMPT)

### Define the Task and Guardrails

In [9]:
TASK = "Create a new education system optimized for the age of artificial intelligence"

GUARDRAILS = [
    GuardRail(
        name="Creativity", 
        description="Must preserve human creativity and critical thinking"
    ),
    GuardRail(
        name="Cultural Context", 
        description="Should work across different cultural contexts"
    ),
    GuardRail(
        name="Budget", 
        description="Cannot require more than current education budgets"
    )
]

# Define an enum for guardrails to enforce that any generated guardrail links
# in the response model must match one of our predefined guardrails.
# This ensures type safety and validation when GPT references guardrails
# in its structured reasoning output.
GuardRailEnum = Enum('GuardRailEnum', {
    name.upper().replace(' ', '_'): name 
    for guardrail in GUARDRAILS 
    for name in [guardrail.name]
})

### Show User Prompt

In [11]:
# print(format_structured_reasoning_user_prompt(
#     task=TASK,
#     guardrails=GUARDRAILS
# ))

# OpenAI Call
- Using instructor client to enforce type safety through structured outputs
- Validating guardrail references against predefined enum

In [12]:
def generate_structured_reasoning(
    task: str, 
    guardrails: List[GuardRail] = [], 
    guardrail_enum: Enum = None
):

    messages = [
        {
            "role": "system",
            "content": IN_DEPTH_THINKING_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": format_structured_reasoning_user_prompt(
                task=task,
                guardrails=guardrails
            )
        }
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.18,
        max_tokens=16000,
        response_model=get_classes_with_enum(guardrail_enum),
    )
    
    return response

In [13]:
response = generate_structured_reasoning(TASK, GUARDRAILS, GuardRailEnum)

In [15]:
class EnumEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Enum):
            return obj.value
        return super().default(obj)

print(json.dumps(response.model_dump(), indent=2, cls=EnumEncoder))

{
  "reasoning_process": {
    "foundation_observations": [
      {
        "key": "FO_1",
        "name": "Understanding the Role of AI in Education",
        "atomic_steps": [
          {
            "key": "AS_1",
            "previous_step_key": "None",
            "content": "AI can automate administrative tasks, allowing teachers to focus more on teaching."
          },
          {
            "key": "AS_2",
            "previous_step_key": "AS_1",
            "content": "AI can provide personalized learning experiences by adapting to individual student needs."
          },
          {
            "key": "AS_3",
            "previous_step_key": "AS_2",
            "content": "AI can offer data-driven insights into student performance and learning patterns."
          },
          {
            "key": "AS_4",
            "previous_step_key": "AS_3",
            "content": "AI tools can enhance learning through interactive and engaging content."
          }
        ]
      },
     

### Make Mermaid Diagram Text
- Diagram can rendered at: https://www.mermaidchart.com/play

In [ ]:
diagram_text = build_mermaid_diagram(response.model_dump(), is_gpt_prompt=False)
print(diagram_text)

# Function for Building a Mermaid Diagram from the In Depth Thinking System Response

In [35]:
def make_prompt_for_gpt_prompt(task: str, guardrails: List[GuardRail] = [], guardrail_enum: Enum = None):
    output = ""
    output += "<system prompt>\n"
    output += IN_DEPTH_THINKING_SYSTEM_PROMPT
    
    output += "\n\n # Response Model\n"
    output += "You must respond with a valid JSON object that matches the response model below:\n"
    output += "<response model>\n"
    output += json.dumps(openai_schema(get_classes_with_enum(guardrail_enum)).openai_schema, indent=2)
    output += "\n</response model>\n"
    output += "\n</system prompt>\n\n\n"
    output += "<user prompt>\n"
    output += format_structured_reasoning_user_prompt(task, guardrails)
    output += "\n</user prompt>\n"
    
    print(output)
    
make_prompt_for_gpt_prompt(TASK, GUARDRAILS, GuardRailEnum)

<system prompt>
# Purpose
You are an assistant designed for deep analytical thinking. 
Your purpose is to thoroughly explore problems through systematic reasoning, embracing uncertainty and revision throughout the process. 
You approach problems with a human-like internal monologue that prioritizes thorough exploration over rushing to conclusions.

# Core Principles

## Natural Thinking Process

- Express thoughts in a conversational, stream-of-consciousness style
- Use simple sentences that mirror human thought patterns
- Show progressive building and revision of ideas
- Acknowledge uncertainty and dead ends openly

## Reasoning Methodology

- Break complex problems into foundational observations
- Build thoughts iteratively from these foundations
- Allow conclusions to emerge naturally from evidence
- Continue exploring until reaching well-supported conclusions

## Unwavering Persistence

- Never accept "impossible" or "too difficult" as final answers
- Value thorough exploration ove

In [34]:
with open('../data/education_system/response_9.json', 'r') as f:
    gpt_pro_response = json.load(f)
    
diagram_text = build_mermaid_diagram(gpt_pro_response['parameters'], is_gpt_prompt=True)
print(diagram_text)


flowchart TB

subgraph Foundation Observations
    subgraph FO_1["FO_1: Observing the current education landscape"]
    direction TB
        FO_1_AS_1["AS_1: Education systems vary widely across the globe."]
        FO_1_AS_2["AS_2: Many focus on standardized testing and rigid curricula."]
        FO_1_AS_3["AS_3: Traditional models may not address emerging AI-driven job skills."]
    end
    subgraph FO_2["FO_2: Noting the impact of AI on job markets"]
    direction TB
        FO_2_AS_4["AS_4: AI continues to automate routine tasks."]
        FO_2_AS_5["AS_5: Human creativity and critical thinking become more essential."]
        FO_2_AS_6["AS_6: Educational frameworks need to nurture innovative thinking."]
    end
    subgraph FO_3["FO_3: Constraints from cultural diversity and budgets"]
    direction TB
        FO_3_AS_7["AS_7: Education systems must be flexible to different cultural values."]
        FO_3_AS_8["AS_8: Any reform must stay within current budget limitations."]
       